<a href="https://colab.research.google.com/github/omkarsk98/NewsDataClassification/blob/development/NewsDataClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/omkarsk98/NewsDataClassification.git

fatal: destination path 'NewsDataClassification' already exists and is not an empty directory.


In [2]:
%cd NewsDataClassification

/content/NewsDataClassification


In [3]:
!git checkout development

Already on 'development'
Your branch is up to date with 'origin/development'.


In [4]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-10-09 10:05:22--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.114.21
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.114.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  57.9MB/s    in 25s     

2019-10-09 10:05:48 (62.5 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [5]:
!gunzip GoogleNews-vectors-negative300.bin.gz

gzip: GoogleNews-vectors-negative300.bin already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [6]:
!ls

GoogleNews-vectors-negative300.bin     NewsDataClassification.ipynb
GoogleNews-vectors-negative300.bin.gz  News_Final.csv
index.py			       README.md


In [7]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
import pandas as pd
from gensim import models
from sklearn.linear_model import LogisticRegression
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import sys

In [9]:
# read csv
main_data = pd.read_csv('News_Final.csv')
# read titles from it
article_titles = main_data['TITLE']
labels = main_data["CATEGORY"]

# Create a list of strings, one for each title
titles_list = [title for title in article_titles]
# form a single string fro the list of strings
big_title_string = ' '.join(titles_list)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
# Tokenize the string into words
tokens = word_tokenize(big_title_string)

# Remove non-alphabetic tokens, such as punctuation
words = [word.lower() for word in tokens if word.isalpha()]
stop_words = set(stopwords.words('english'))
words = [word for word in words if not word in stop_words]

In [11]:
# Load word2vec model (trained on an enormous Google corpus)
model = models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True) 
# Check dimension of word vectors
print("Dimensions of the model",model.vector_size)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Dimensions of the model 300


In [0]:
# Using some pre built functions
def document_vector(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in model.vocab]
    return np.mean(model[doc], axis=0)

# Our earlier preprocessing was done when we were dealing only with word vectors
# Here, we need each document to remain a document 
def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    doc = [word for word in doc if word.isalpha()] 
    return doc

# Function that will help us drop documents that have no word vectors in word2vec
def has_vector_representation(word2vec_model, doc):
    """check if at least one word of the document is in the
    word2vec dictionary"""
    return not all(word not in word2vec_model.vocab for word in doc)

# Filter out documents
def filter_docs(corpus, texts, labels, condition_on_doc):
    """
    Filter corpus and texts given the function condition_on_doc which takes a doc. The document doc is kept if condition_on_doc(doc) is true.
    """
    number_of_docs = len(corpus)
    
    if texts is not None:
        texts = [text for (text, doc) in zip(texts, corpus)
                 if condition_on_doc(doc)]
    

    corpus = [doc for doc in corpus if condition_on_doc(doc)]
    
    final_labels = []
    for i in range(len(corpus)):
      if condition_on_doc(corpus[i]):
        final_labels.append(labels[i])
    
    print("{} docs removed".format(number_of_docs - len(corpus)))

    return (corpus, texts, final_labels)

In [13]:
# # Preprocess the corpus to get list of documents with stop words removed and containing only the words that are present in the vocab
corpus = [preprocess(title) for title in titles_list]
# # still contains all the dcuments, nothing is filtered
# # corpus[0] => ['fed', 'official', 'says', 'weak', 'data', 'caused', 'weather', 'slow', 'taper']
# # corpus[1] => ['fed', 'charles', 'plosser', 'sees', 'high', 'bar', 'change', 'pace', 'tapering']

# # Remove docs that don't include any words in W2V's vocab
corpus, titles_list, labels = filter_docs(corpus, titles_list, labels, lambda doc: has_vector_representation(model, doc))
print("1st filter",len(corpus), len(titles_list), len(labels))
# # corpus[0] => ['fed', 'official', 'says', 'weak', 'data', 'caused', 'weather', 'slow', 'taper']
# # corpus[1] => ['fed', 'charles', 'plosser', 'sees', 'high', 'bar', 'change', 'pace', 'tapering']
# # titles_list[0] => Fed official says weak data caused by weather, should not slow taper
# # titles_list[1] => Fed's Charles Plosser sees high bar for change in pace of tapering


# # Filter out any empty docs
corpus, titles_list, labels = filter_docs(corpus, titles_list, labels, lambda doc: (len(doc) != 0))
print("2nd filter",len(corpus), len(titles_list), len(labels))
# # corpus[0] => ['fed', 'official', 'says', 'weak', 'data', 'caused', 'weather', 'slow', 'taper']
# # titles_list[0] => Fed official says weak data caused by weather, should not slow taper

x = []
for doc in corpus: # append the vector for each document
    x.append(document_vector(model, doc))
    
vectorsForEachDocument = np.array(x) # list to array
labels = np.array(labels)
len(vectorsForEachDocument)

241 docs removed
1st filter 422178 422178 422178
0 docs removed
2nd filter 422178 422178 422178


422178

# **What all does the data contain at the moment.**

---
corpus: A list containing a list of words of which stop words are removed. Only the words that are present in the vocab are present here.<br>
titles_list: It contains a list of all titles.<br>
vectorsForEachDocument: It contains vectors for each document. This attribute can be used as the feature.<br>
labels: Contains the category for respective title. This attribute can be used as the label.<br>

In [14]:
# vectorsForEachDocument.shape
labels = labels.reshape(len(labels),1)
labels.shape

(422178, 1)

# Split the data
---
**Train Data**: Use 80% of the data for training purpose. <br>
**Test Data**: Use 20% of the data for testing purpose. <br>
**Features**: Use 300 dimensional vectors as features. It can be found in `vectorsForEachDocument`.<br>
**Labels**: Use the categories as labels. I can be found in `labels`.<br>

In [15]:
train = int((80/100)*len(vectorsForEachDocument))
train = 9000
test = 1000
trainFeatures, testFeatures = vectorsForEachDocument[:train], vectorsForEachDocument[train:train+test]
trainLabels, testLabels = labels[:train], labels[train:train+test]
trainFeatures.shape, trainLabels.shape, testFeatures.shape, testLabels.shape

((337742, 300), (337742, 1), (84436, 300), (84436, 1))

**Lengths of Training and test data** <br>
Training Set contains 337742 records.<br>
Test Set contains 84436 records.<br><br>
**Shapes of the data** <br>
trainFeatures: (337742,300) <br>
trainLabels: (337742,1) <br>
testFeatures: (84436,300) <br>
testLabels: (84436,1) <br>

In [0]:
# tic = sys.time()
logistic_Regression = LogisticRegression()
logistic_Regression.fit(trainFeatures,trainLabels)
Y_predict = logistic_Regression.predict(testFeatures)
print(accuracy_score(testLabels,Y_predict))
# toc = sys.time()
# print("Time taken:"+str((toc-tic)*1000)+"ms")

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
